# PE1 - Ein generisches Perzeptron

Um mit dem Perzeptron experimentieren zu können, implementieren wir es als generische Klasse, in der die Anzahl der Eingabe und Ausgabe-Neuronen beliebig definiert werden kann. Hier noch einmal die wichtigsten Eigenschaften:

__Perzeptron-Regel (mit Bias)__ Der (binäre) Wert des Output-Neurons $o_j$ ergibt sich als gewichtete Summe der $N$ Input-Neuronen:

$$
o_j(x) = \left\{
\begin{array}{ll}
1 & \sum_{k=1}^N w_{jk} x_k + b> 0 \\
0 & \sum_{k=1}^N w_{jk} x_k + b\leq 0 \\
\end{array}
\right. 
$$

Der __Lernalgorithmus__ bleibt erhalten:

$$
\begin{equation}
\begin{split}
w_{jk} &\rightarrow w_{jk} + \alpha (t_j - o_j) x_{k}\\
b &\rightarrow b + \alpha (t -o )
\end{split}
\end{equation}
$$
Dabei kann der Ausdruck $(t_j - o_j)$, d.h. die Differenz zwischen erwartetem (Target-)Wert $t$ und Outputwert $o$, jeweils nur die Werte $\pm 1$ und 0 annehmen.

Was bedeutet der Ausdruck $w_{jk} + \alpha (t_j - o_j) x_{k}$? Die $w_{jk}$ bilden Gewichtsmatrix, die die Gewichte aller Verbindungen von den Eingabe-Nodes $x_k$  zu den Ausgabe-Nodes $o_j$ enthält. Bei $K$ Eingabe-Nodes und $J$ Ausgabe-Nodes ist dies eine $K \times J$-Matrix. Der Fehler $(t_j - o_j)$ bildet einen Vektor mit $J$ Komponenten, und die Eingabe-Nodes einen Vektor mit $K$ Komponenten. Wir müssen also aus dem Fehlervektor und dem Eingabevektor eine Matrix derselben Gestalt wie die Gewichtsmatrix bilden, d.h. jedes Element des Fehlervektors mit jedem Element des Eingabevektors _kombinieren_. Wir haben bereits das __Skalarprodukt__ kennengelernt, mit dem aus zwei Vektoren ein Skalar gebildet wird. Hier wollen wir nun aus zwei Vektoren eine Matrix erzeugen, und die dazugehörende mathematische Operation wird das __äußere Produkt__ oder __Tensorprodukt__ genannt (das Skalarprodukt wird dagegen auch als __inneres Produkt__ bezeichnet). Sind $W$ die Gewichtsmatrix und $(t-o)$ und $x$ die Fehler- bzw. Eingabevektoren, so schreibt sich die Lernregel kurz:

$$ W \rightarrow W + \alpha (t-o) \otimes x$$

Man könnte nun mit zwei for-Schleifen über $j$ und $k$ die Komponenten einzeln bestimmen. NumPy unterstützt diese Operation aber wieder direkt mit einer eigenen Funktion namens __outer__. Damit können wir nun unser Perzeptron implementieren

In [1]:
import numpy as np

In [2]:
class Perceptron:
    """ Ein generisches Perzeptron """
    
    def __init__(self,*nodes: list[int]) -> None:
        """ Initialisert das Perzeptron. Gewichte und Bias werden auf zufällige Werte zwischen -1 und +1 gesetzt. """
        input, output = nodes
        self.w = np.random.uniform(-1,1,(input,output)).T
        self.b = np.random.uniform(-1,1,output)    
        pass

    def train(self,input: list[int],target: list[int],alpha:float = 0.1,epochs: int = 1) ->  None:
        """ 
        Trainiert das Perzeptron mit den übergebenen Daten input und den Zielwerten target gemäß dem Lernalgorithmus. 
        alpha bestimmt die Lernrate und epochs die Anzahl der Durchläufe. 
        """
        for _ in range(epochs):
            for x, t in zip(input,target):  
                error = (t - self.forward(x))     
                self.w += alpha * np.outer(error,x)
                self.b += alpha * error
        pass

    def forward(self,x: list[int]) -> list[int]:
        """ 
        Berechnet das Ergebnis bei einem angelegten Input x. 
        Durch die Aktivierungsfunktion sind nur die Werte 0 (inaktiv) und 1 (aktiv) möglich.
        """
        return self.activation(np.dot(self.w,x) + self.b) 

    def activation(self,signal: list[int]) -> int:
        """ Die Aktivierungsfunktion testet, obL das Signal positiv ist (Perzeptron-Regel). """
        return (signal > 0).astype(int)    
   
    def test(self,input: list[int],target: list[int]) -> list[bool]:
        """ 
        Testet die Funktion des Perzeptrons, indem alle durch die Eingabewerte input erzeugten Ausgaben mit den 
        Zielwerten target verglichen werden
        """
        y = self.forward(input)
           
    def __str__(self) -> str:
        return "Gewichte: {0}, Bias = {1}".format(self.w, self.b)


## Anwendung des Perzeptrons

Wir probieren unser generisches Perzeptron wieder an den logischen Schaltungen aus.

In [3]:
x = np.array([[x,y] for x in [0,1] for y in [0,1]])

logic_functions = {"AND": lambda a,b : a and b, 
                   "OR": lambda a,b : a or b, 
                   "NAND": lambda a,b : not (a and b), 
                   "XOR": lambda a,b : (a and not b) or (not a and b)}

for n,f in logic_functions.items():
    t = [[f(a,b)] for (a,b) in x]  
    p = Perceptron(2,1)
    p.train(x,t,epochs = 100)
    print(f'--- {n} ----')
    for tx in x:    
        print(f'{tx} -> {p.forward(tx)}')   

--- AND ----
[0 0] -> [0]
[0 1] -> [0]
[1 0] -> [0]
[1 1] -> [1]
--- OR ----
[0 0] -> [0]
[0 1] -> [1]
[1 0] -> [1]
[1 1] -> [1]
--- NAND ----
[0 0] -> [1]
[0 1] -> [1]
[1 0] -> [1]
[1 1] -> [0]
--- XOR ----
[0 0] -> [1]
[0 1] -> [0]
[1 0] -> [0]
[1 1] -> [0]


Das Ergebnis ist wieder dasselbe, incl. XOR-Problem.

### Links

Weitere Details findet man im Wikipedia-Artikel zum Perzeptron: https://de.wikipedia.org/wiki/Perzeptron